In [1]:
import torch
import numpy as np
import gym

In [2]:
from tensorboardX import SummaryWriter

In [8]:
## dqn is q-leanring by nn
##which is model-free off-policy td method value-based: Qfunction of state and action
## two nn; one for behave network, one for target network
## memory replay: to get iid data
## Q(s,a) = Q(s,a) + a*(r+ discount* maxQ(s',a')- Q(s,a))
## target network to get Q(s',a'); otherwisw, if Q(s',a') from behave policy, the updated of   behave policy  may hurt Q(s',a') 

# 1 nn  model

In [10]:
class DQN(torch.nn.Module):
    '''
    dqn model for q function network and target network
    '''
    def __init__(self,input_shape,n_actions):
        super(DQN,self).__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_shape[0],out_channels=32,kernel_size=8,stride=4),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=4,stride=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1),
            torch.nn.ReLU()
        )
        conv_out_size = self.__get_conv_out(input_shape) ##计算输出size
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=conv_out_size,out_features=512),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=512,out_features=n_action)
        )
    
    def __get_conv_out(self,input_shape):
        ##使用假数据，得到conv的输出size
        o = self.conv(torch.zeros(1,*shape))
        return int(np.prod(o.size()))
        
    def forward(self,x):
        conv_out = self.conv(x).view(x.size()[0],-1)
        return self.fc(conv_out)

In [9]:
(84-8+1)/4

19.25

In [12]:
## step
## step1: get action by epsilon-greddy ,get state,action,reward,next_state
## step2: stored state,action,reward,next_sate to memory
## step3: sample state,action,reward,next_state from memory
## step4: calc Q(s,a) from Q_net and calc maxQ'(s',a') from target_net
## step5 :calc loss by r + discount*maxQ(s',a') with Q(s,a) to update Q_net
## repeat